# Importing Libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score
import xgboost as xgb
from sklearn.metrics import roc_curve, auc
import gradio as gr
from sklearn.utils import class_weight
from sklearn.metrics import precision_score, recall_score, f1_score

#  Importing CSV Data

In [6]:
dataset = pd.read_csv('C:/Users/Lenovo/OneDrive/Desktop/project/parkinson.csv')
x=dataset.loc[:,dataset.columns!='status'].values[:,1:]
x1=dataset.loc[:,dataset.columns!='status']
y=dataset.loc[:,'status'].values
y1=dataset.loc[:,'status']

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Lenovo/OneDrive/Desktop/project/parkinson.csv'

# Exploratory Data Analysis(EDA)

In [ ]:
dataset.describe()

In [ ]:
dataset.shape

In [ ]:
dataset.head(10)

In [ ]:
column_name = 'status'
column_index = dataset.columns.get_loc(column_name)

print('Index of column {} is {}'.format(column_name, column_index))

In [ ]:
dataset.rename(columns=({'MDVP:Fo(Hz)':'avg_fre', 'MDVP:Fhi(Hz)':'max_fre', 'MDVP:Flo(Hz)':'min_fre', 'MDVP:Jitter(%)':'var_fre1',
       'MDVP:Jitter(Abs)':'var_fre2', 'MDVP:RAP':'var_fre3', 'MDVP:PPQ':'var_fre4', 'Jitter:DDP':'var_fre5',
       'MDVP:Shimmer':'var_amp1', 'MDVP:Shimmer(dB)':'var_amp2', 'Shimmer:APQ3':'var_amp3', 'Shimmer:APQ5':'var_amp4',
       'MDVP:APQ':'var_amp5', 'Shimmer:DDA':'var_amp6'}),inplace=True)
dataset


In [ ]:
dataset.info()

In [ ]:
dataset.duplicated().sum()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.columns

In [ ]:
dataset.skew()
#calculating the skewness of all parameters

In [ ]:
x1.hist(figsize=(25,20))
#plotting histogram, width = 25  height = 20 
plt.show()

In [ ]:
#Filter data
dataset[dataset['status']==1].head()

In [ ]:
dataset.corr()
#calculating the correlation of our dataset

In [ ]:
#Define plot size
plt.figure(figsize=[6,6])

#Define column to use
data = dataset["status"].value_counts(normalize=True)

#Define labels
labels = ["Have PD","Doesnot have PD",]

#Define color palette
colors = sns.color_palette('pastel')

#Create pie chart
plt.pie(data,labels=labels,colors=colors, autopct='%.0f%%')
plt.title("Proportion of Patients by Group")
plt.show()

In [ ]:
dataset["status"].value_counts()

In [ ]:
correl=x1.corr()
#plotting a heatmap
plt.figure(figsize=(20,20))
#size of heatmap = 20, 20
sns.heatmap(correl,annot=True,cmap='OrRd')
plt.show()

# Traning Test Data Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
len(y_train)

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#fitting our training and test set into the object sc 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 0,class_weight='balanced')
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
lr.score(X_test,y_test)*100
#calculating accuracy

In [ ]:

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr.classes_)
disp.plot()
plt.show()


In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output=lr.predict(newinput)
output

In [ ]:
y_predict = lr.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_predict)

roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
# lw =2; width of the line plot is set to 2 points
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
#setting limit of x-asis from 0.0 to 1.0
plt.ylim([0.0, 1.05])
#setting limit of y-axis from 0.0 to 1.5
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# XGBOOST

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 0,class_weight='balanced')
xgb.fit(X_train, y_train)

In [ ]:
y_pred= xgb.predict(X_test)
y_pred

In [ ]:
len(y_pred)

In [ ]:
y_test

In [ ]:
xgb.score(X_test,y_test)*100

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb.classes_)
disp.plot()
plt.show()


In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output= xgb.predict(newinput)
output


In [ ]:
y_predict = xgb.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_predict)

roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Support Vector Classifier(SVC)

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf',degree=1, random_state = 0,class_weight='balanced',C= 12, gamma= 0.1)
classifier.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
y_pred=classifier.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
classifier.score(X_test,y_test)*100

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classifier.classes_)
disp.plot()
plt.show()


In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output=classifier.predict(newinput)
output

In [ ]:

classifier.fit(X_train, y_train)
y_predi = classifier.decision_function(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_predi)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)


In [ ]:
y_test

In [ ]:
y_pred=gnb.predict(X_test)
y_pred

In [ ]:
gnb.score(X_test,y_test)*100

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gnb.classes_)
disp.plot()
plt.show()


In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output=gnb.predict(newinput)
output

In [ ]:
y_predict = gnb.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_predict)


roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0,class_weight='balanced')
dt.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
y_pred=dt.predict(X_test)
y_pred

In [ ]:
dt.score(X_test,y_test)*100

In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output=dt.predict(newinput)
output

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt.classes_)
disp.plot()
plt.show()


In [ ]:
y_predict = dt.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_predict)


roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rclassifier = RandomForestClassifier(random_state = 0,class_weight='balanced',bootstrap= True, criterion='entropy', max_depth= None, max_features='log2', min_samples_leaf= 9, min_samples_split= 6, n_estimators= 24)
rclassifier.fit(X_train, y_train)

In [ ]:
y_test

In [ ]:
y_pred=rclassifier.predict(X_test)
y_pred

In [ ]:
rclassifier.score(X_test,y_test)*100

newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rclassifier.classes_)
disp.plot()
plt.show()


In [ ]:
output=rclassifier.predict(newinput)
output

In [ ]:
y_predict = rclassifier.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_predict)


roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# K-Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kclassifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
kclassifier.fit(X_train, y_train)

In [ ]:
y_pred=kclassifier.predict(X_test)
y_pred

In [ ]:
kclassifier.score(X_test,y_test)*100

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
#calculating precision, recall and f1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=kclassifier.classes_)
disp.plot()
plt.show()


In [ ]:
newinput=[[274.688,240.005,174.287,0.01360,0.01008,0.00624,0.00564,0.01873,1.02308,0.256,0.51268,0.01365,0.81667,0.63804,0.10715,6.883,0.607567,0.158453,3.679772,0.131728,2.381826,0.206552]]

In [ ]:
output=kclassifier.predict(newinput)
output

In [ ]:
y_predict = kclassifier.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_predict)


roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

# Gradio Library

In [ ]:
#geting our columns

print(dataset.columns)

In [ ]:
def pd(avg_fre, max_fre, min_fre, var_fre1, var_fre2,var_fre3, var_fre4, var_fre5, var_amp1, var_amp2, var_amp3,
       var_amp4, var_amp5, var_amp6, NHR, HNR,RPDE,DFA, spread1, spread2, D2, PPE):
#turning the arguments into a numpy array  

 x = np.array([avg_fre, max_fre, min_fre, var_fre1, var_fre2,var_fre3, var_fre4, var_fre5, var_amp1, var_amp2, var_amp3,
       var_amp4, var_amp5, var_amp6, NHR, HNR,RPDE,DFA, spread1, spread2, D2, PPE])

 output=kclassifier.predict(x.reshape(1, -1))
 if output==1:
        return("You may have PD.Plese consult doctor")
 elif output==0:
  return("You may not have PD")


In [ ]:
outputs = gr.outputs.Textbox()

app = gr.Interface(fn=pd, inputs=['number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number','number'], outputs=outputs,description="Parkinson's Disease Prediction using K-Nearest Neighbour")

In [161]:
app.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [162]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


# Define parameter grid to search over
param_grid = {
    'C': [0.1, 1,5, 10, 12],
    'gamma': [0.001, 0.01, 0.1],
    'kernel': ['linear', 'rbf', 'poly']
}
#param_grid is a dictionary of all hyper parameters

#c determines the margin between support vectors and decision boundary ; c inversely proportional to margin
#gamma determines the shape of decision boundary, small value of gamma means smooth boundary and vice versa
#kernel determines the type of deicision boundary to be used


svm = SVC()

# Use grid search to find optimal hyperparameters
grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5)
#cv = 5 ; 5 folds of cross validation
grid_search.fit(X_train, y_train)

# Print best hyperparameters and score
print("Best hyperparameters:", grid_search.best_params_)
#prints the most feasible parameters
print("Best score:", grid_search.best_score_)

Best hyperparameters: {'C': 12, 'gamma': 0.1, 'kernel': 'rbf'}
Best score: 0.9454545454545455


In [163]:

from sklearn.datasets import load_iris
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint


iris = load_iris()
x, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

rfc = RandomForestClassifier()

# Define the hyperparameters to search over and their distributions
param_dist = {'n_estimators': randint(10, 100),
              'max_depth': [3, 5, 10, None],
              'max_features': ['sqrt', 'log2'],
              'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 10),
              'bootstrap': [True, False],
              'criterion': ['gini', 'entropy']}

#n_estimators: The number of trees in the forest
#max_depth: The maximum depth of each decision tree in the forest
#max_features: The number of features to consider when looking for the best split
#min_samples_split: The minimum number of samples required to split an internal node
#bootstrap: Whether to use bootstrapping when building the trees, gives boolean value
#criterion: The function to measure the quality of a split

# Set up the randomized search
rs = RandomizedSearchCV(estimator=rfc, param_distributions=param_dist, cv=5, n_iter=100, random_state=42, n_jobs=-1)
#n_iter = 100; 100 different combinations of hyperparameters will be sampled and evaluated
#n_jobs: The number of CPU cores to use for parallel processing.  n_jobs=-1 specifies that all available CPU cores will be used.

rs.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print(f"Best hyperparameters: {rs.best_params_}")
print(f"Best score: {rs.best_score_:.3f}")

Best hyperparameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 35}
Best score: 0.967
